In [51]:

import pandas as pd
import os
extracted_train_folder = 'extracted_train_data_files'
extracted_annonimized_folder = 'extracted_annonimized_files'
extracted_train_files = os.listdir(extracted_train_folder)
extracted_annonimized_files = os.listdir(extracted_annonimized_folder)
# Load the training data files
train_file_paths = [os.path.join(extracted_train_folder, file) for file in extracted_train_files]
train_dfs = {os.path.basename(path): pd.read_csv(path) for path in train_file_paths}

# Load the annonimized data
annonimized_df = pd.read_csv('extracted_annonimized_files/annonimized.csv')

# Load the student data
student_data_path = 'student.csv'
student_df = pd.read_csv(student_data_path)

# Display the first few rows of each DataFrame
for file_name, df in train_dfs.items():
    print(f"Data from {file_name}:")
    print(df.head(), "\n")

print("Annonimized data:")
print(annonimized_df.head(), "\n")

print("Student data:")
print(student_df.head(), "\n")

Data from ck-public.csv:
                                 student_id   CK
0  00b6dd4fc7eb817e03708c532016ef30ce564a61  5.5
1  00bef8afee8f3c595d535c9c03c490cac1a4f021  6.5
2  01122b3ef7e59b84189e65985305f575d6bdf83c  4.5
3  013de369c439ab0ead8aa7da64423aa395a8be39  5.0
4  014c59c6433fd764a0b08de6ffeb757eaf60aa73  4.0 

Data from qt-public.csv:
                                 student_id diemqt
0  00b6dd4fc7eb817e03708c532016ef30ce564a61    7.5
1  00bef8afee8f3c595d535c9c03c490cac1a4f021      7
2  01122b3ef7e59b84189e65985305f575d6bdf83c      9
3  013de369c439ab0ead8aa7da64423aa395a8be39     10
4  014c59c6433fd764a0b08de6ffeb757eaf60aa73      9 

Data from tbtl-public.csv:
                                 student_id  TBTL
0  00b6dd4fc7eb817e03708c532016ef30ce564a61  7.24
1  00bef8afee8f3c595d535c9c03c490cac1a4f021  8.11
2  01122b3ef7e59b84189e65985305f575d6bdf83c  7.30
3  0134f9f410c65ad0e8c2254a7e9288670e02a183  8.63
4  013de369c439ab0ead8aa7da64423aa395a8be39  8.20 

Data from th-publ

In [52]:
print(len(train_dfs))

4


In [225]:
# Convert submission and evaluation dates to datetime
annonimized_df = pd.read_csv('extracted_annonimized_files/annonimized.csv')

# annonimized_df['created_at'] = '2023-' + str(annonimized_df['created_at'])
# annonimized_df['updated_at'] = '2023-' + str(annonimized_df['updated_at']) 
# annonimized_df['created_at'] = pd.to_datetime(annonimized_df['created_at'], errors='ignore')
# annonimized_df['updated_at'] = pd.to_datetime(annonimized_df['updated_at'], errors='ignore')

# annonimized_df['submission_delay'] = (annonimized_df['updated_at'] - annonimized_df['created_at']).dt.total_seconds()
annonimized_df=annonimized_df.drop(columns=['problem_id','language_id'],axis=1)
# Aggregate features for each student
student_agg = annonimized_df.groupby('student_id').agg(
    total_submissions=('assignment_id', 'count'),
    avg_testcase_passed=('pre_score', 'mean'),
    max_testcase_passed=('pre_score', 'max'),
    min_testcase_passed=('pre_score', 'min'),
    scored_submissions=('is_final', lambda x: (x == 1).sum()),
    late_submissions=('coefficient', lambda x: (x < 100).sum())
).reset_index()
student_agg.to_csv('all_student3.csv',index=False)

C:\Users\APTS\AppData\Local\Temp\ipykernel_17288\562458736.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  annonimized_df['created_at'] = pd.to_datetime(annonimized_df['created_at']).dt.date


OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 10-09 08:02:04, at position 0

In [246]:
df=pd.read_csv('all_student3.csv')
# Assuming `df` is your DataFrame containing the original features
df['success_rate'] = df['avg_testcase_passed'] / df['total_submissions']
df['scored_to_total_ratio'] = df['scored_submissions'] / df['total_submissions']
df['testcase_range'] = df['max_testcase_passed'] - df['min_testcase_passed']
df['total_testcases_passed'] = df[['avg_testcase_passed', 'max_testcase_passed', 'min_testcase_passed']].sum(axis=1)
df.to_csv('all_student2.csv',index=False)

In [247]:
all_student=pd.read_csv('all_student2.csv')
student_train=pd.read_csv('train_student.csv')
student_train=all_student.merge(student_train,on='student_id',how='right')
student_train.to_csv('student_train_with_features.csv',index=False)

In [248]:
all_student=pd.read_csv('all_student2.csv')
student_test=pd.read_csv('student_test.csv')
student_train=all_student.merge(student_test,on='student_id',how='right')
student_train.to_csv('student_test_with_features.csv',index=False)

In [249]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Corrected: Read the CSV file into a DataFrame
all_student_train = pd.read_csv('student_train_with_features.csv')
all_student_test = pd.read_csv('student_test_with_features.csv')

# Instantiate the LabelEncoder
encoder = LabelEncoder()

# Assuming you want to encode the 'some_column' column
# Replace 'some_column' with the actual column name you want to encode
all_student_train['student_id'] = encoder.fit_transform(all_student_train['student_id'])
all_student_test['student_id'] = encoder.fit_transform(all_student_test['student_id'])
all_student_train.to_csv('student_train_with_features_final.csv',index=False)
all_student_test.to_csv('student_test_with_features_final.csv',index=False)
# Now 'all_student' DataFrame has a new column 'some_column_encoded' with encoded values

In [250]:
import pandas as pd

# Combine the training data into a single DataFrame if necessary
student_train=pd.read_csv('student_train_with_features.csv')
ck=pd.read_csv('extracted_train_data_files\ck-public.csv')
student_train_score=student_train.merge(ck,on='student_id',how='inner')
student_train_score.to_csv('student_train_with_score.csv',index=False)
# for df in train_dfs.values():
#     student_train=student_train.merge(df,on='student_id',how='outer')
# student_train.to_csv('student_train_with_score.csv',index=False)

In [251]:
import pandas as pd

# Combine the training data into a single DataFrame if necessary
student_train=pd.read_csv('student_train_with_score.csv')
qt=pd.read_csv('extracted_train_data_files\qt-public.csv')
student_train_score=student_train.merge(qt,on='student_id',how='inner')
student_train_score.to_csv('student_train_with_score.csv',index=False)
# for df in train_dfs.values():
#     student_train=student_train.merge(df,on='student_id',how='outer')
# student_train.to_csv('student_train_with_score.csv',index=False)

In [252]:
import pandas as pd

# Combine the training data into a single DataFrame if necessary
student_train=pd.read_csv('student_train_with_score.csv')
tbtl=pd.read_csv('extracted_train_data_files/tbtl-public.csv')
student_train_score=student_train.merge(tbtl,on='student_id',how='inner')
student_train_score.to_csv('student_train_with_score.csv',index=False)
# for df in train_dfs.values():
#     student_train=student_train.merge(df,on='student_id',how='outer')
# student_train.to_csv('student_train_with_score.csv',index=False)

In [253]:
import pandas as pd

# Combine the training data into a single DataFrame if necessary
student_train=pd.read_csv('student_train_with_score.csv')
th=pd.read_csv('extracted_train_data_files/th-public.csv')
student_train_score=student_train.merge(th,on='student_id',how='inner')
student_train_score.to_csv('student_train_with_score.csv',index=False)
# for df in train_dfs.values():
#     student_train=student_train.merge(df,on='student_id',how='outer')
# student_train.to_csv('student_train_with_score.csv',index=False)

In [254]:
import numpy as np
# Merge with the aggregated annonimized data
student_train=pd.read_csv('student_train_with_score.csv')
feature=['diemqt','TH']
student_train[feature]=student_train[feature].replace('\xa0', 5)

# Handle missing values
student_train.fillna(5, inplace=True)

# Feature engineering (adjust based on actual features)
# Example: Create new features or transform existing ones if necessary

student_train.to_csv('training.csv',index=False)


In [255]:
features = ['total_submissions', 'avg_testcase_passed', 'max_testcase_passed', 'min_testcase_passed', 'scored_submissions', 'late_submissions','success_rate','scored_to_total_ratio','testcase_range','total_testcases_passed']
targets = ['CK', 'diemqt', 'TBTL', 'TH']

In [257]:
merged_df = pd.read_csv('training.csv')
print(merged_df.nunique())

student_id                761
total_submissions         377
avg_testcase_passed       745
max_testcase_passed        11
min_testcase_passed        12
scored_submissions        130
late_submissions           33
success_rate              748
scored_to_total_ratio     663
testcase_range             17
total_testcases_passed    746
CK                         21
diemqt                     20
TBTL                      288
TH                         20
dtype: int64


In [264]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
merged_df = pd.read_csv('training.csv')
# Split training data
X = merged_df[features]
y = merged_df[targets]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestRegressor(n_estimators=200, random_state=400)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
r2 = r2_score(y_test, y_pred, multioutput='uniform_average')
print(f'R^2 Score: {r2}')


R^2 Score: 0.10300496097829748


In [261]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
merged_df = pd.read_csv('training.csv')

X = merged_df[features]
y = merged_df[targets]
# Initialize the model
model = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=452)

# Train the model
model.fit(X_train, y_train)
# Making predictions
y_pred = model.predict(X_test)

# Evaluating the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 3.3079056421558093


In [266]:
# Predict for the student data
student_df=pd.read_csv('student_test_with_features.csv')

student_X = student_df[features]
predictions=model.predict(student_X)
student_predictions = np.round_(predictions*2)/2

# Add predictions to the student DataFrame
student_df[targets] = student_predictions
# Save the predictions
student_df[['student_id', 'TH']].to_csv('result_th.csv', index=False)
print(student_df.head())


                                 student_id  total_submissions  \
0  ed9eaeb6a707f50154024b24d7efcb874a9795dd                139   
1  ba12c0a2cb367af0467e479c03507c71a805d291                319   
2  b7298b0fe50443a623af9b56792b330c2d052845                247   
3  c60be70309789b39355dc612f36e37090ccad5dc                172   
4  a22a58c5be8aa2c2700619e37f2b7a6e4efa7e6b                288   

   avg_testcase_passed  max_testcase_passed  min_testcase_passed  \
0          5305.553957                10000                    0   
1          5228.576803                10000                    0   
2          5998.854251                10000                    0   
3          6237.773256                10000                    0   
4          5547.465278                10000                    0   

   scored_submissions  late_submissions  success_rate  scored_to_total_ratio  \
0                  58                 0     38.169453               0.417266   
1                 117             